## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import gmaps
import requests

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Vacation_Search/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,60.46,36,20,13.80,few clouds
1,1,Norman Wells,CA,65.2820,-126.8329,23.04,86,90,6.91,light snow
2,2,Locri,IT,38.2414,16.2623,66.16,70,100,13.69,overcast clouds
3,3,Yulara,AU,-25.2406,130.9889,59.67,30,61,9.55,broken clouds
4,4,Bohicon,BJ,7.1783,2.0667,75.49,92,100,3.80,overcast clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 100


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Bohicon,BJ,7.1783,2.0667,75.49,92,100,3.80,overcast clouds
5,5,Baculin,PH,7.4525,126.5842,78.13,86,59,4.90,broken clouds
6,6,Atuona,PF,-9.8000,-139.0333,78.28,71,1,17.29,clear sky
8,8,Cockburn Town,TC,21.4612,-71.1419,82.69,67,3,2.73,clear sky
9,9,Porto Nacional,BR,-10.7081,-48.4172,89.60,27,16,0.69,few clouds
12,12,Rio Grande,BR,-32.0350,-52.0986,77.11,78,0,14.34,clear sky
13,13,Rikitea,PF,-23.1203,-134.9692,75.49,80,81,17.65,light rain
18,18,Sao Joao Da Barra,BR,-21.6403,-41.0511,77.09,86,84,11.86,broken clouds
20,20,Vaini,TO,-21.2000,-175.2000,78.96,78,75,9.22,broken clouds
21,21,Sola,VU,-13.8833,167.5500,81.32,76,43,14.50,scattered clouds


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()


City_ID                267
City                   267
Country                264
Lat                    267
Lng                    267
Max Temp               267
Humidity               267
Cloudiness             267
Wind Speed             267
Current Description    267
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [8]:
clean_df.count()

City_ID                264
City                   264
Country                264
Lat                    264
Lng                    264
Max Temp               264
Humidity               264
Cloudiness             264
Wind Speed             264
Current Description    264
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
clean_hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
clean_hotel_df["Hotel Name"] = ""
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Bohicon,BJ,75.49,overcast clouds,7.1783,2.0667,
5,Baculin,PH,78.13,broken clouds,7.4525,126.5842,
6,Atuona,PF,78.28,clear sky,-9.8000,-139.0333,
8,Cockburn Town,TC,82.69,clear sky,21.4612,-71.1419,
9,Porto Nacional,BR,89.60,few clouds,-10.7081,-48.4172,
12,Rio Grande,BR,77.11,clear sky,-32.0350,-52.0986,
13,Rikitea,PF,75.49,light rain,-23.1203,-134.9692,
18,Sao Joao Da Barra,BR,77.09,broken clouds,-21.6403,-41.0511,
20,Vaini,TO,78.96,broken clouds,-21.2000,-175.2000,
21,Sola,VU,81.32,scattered clouds,-13.8833,167.5500,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# 6b. Iterate through the hotel DataFrame.
for index, row in clean_hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        clean_hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")    
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df.tail(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
639,Birao,CF,80.19,broken clouds,10.2849,22.7882,
643,Sibu,MY,73.56,overcast clouds,2.3000,111.8167,RH HOTEL (4 Star)
649,Bagotville,AU,74.64,overcast clouds,-28.9833,153.4167,Broadwater Sunrise Caravan Park
652,Korhogo,CI,79.21,overcast clouds,9.4500,-5.6333,Hôtel Tchinbota
653,Jalu,LY,74.25,few clouds,29.0331,21.5482,صاحبة الفخامة
656,Mto Wa Mbu,TZ,74.77,overcast clouds,-3.3500,35.8500,Fanaka Safaris Campsite & Lodges
657,Grand-Santi,GF,86.04,overcast clouds,4.2500,-54.3833,Hotel La Villa Couacou
659,Mindelo,CV,76.55,scattered clouds,16.8901,-24.9804,Hotel Gaudi
661,Sao Domingos,BR,86.38,broken clouds,-13.3983,-46.3183,Hotel e Restaurante Uirapuru (Márcio uirapuru ...
665,Gorontalo,ID,75.00,scattered clouds,0.5412,123.0595,Maqna Hotel by Prasanthi


In [16]:
# # 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))